In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# === Path to dataset folder ===
base_path = Path(r"D:\recommendation dataset")

# === Find all parquet files ===
all_parquet_files = list(base_path.glob("*.parquet"))
print(f"Found {len(all_parquet_files)} parquet files in the folder.")

# === Classify files by type ===
sales_item_files = []
sales_purchase_files = []
sales_user_files = []

for file_path in all_parquet_files:
    name = file_path.name.lower()
    if "sales_pers.item" in name:
        sales_item_files.append(file_path)
    elif "sales_pers.purchase" in name or "sales_pers.purchase_history_daily" in name:
        sales_purchase_files.append(file_path)
    elif "sales_pers.user" in name:
        sales_user_files.append(file_path)

print(f"""
Summary:
  sales_pers.item: {len(sales_item_files)} files
  sales_pers.purchase: {len(sales_purchase_files)} files
  sales_pers.user: {len(sales_user_files)} files
""")

# === Function to read parquet files ===
def read_parquet_group(file_list, group_name):
    if not file_list:
        print(f"No files for group {group_name}")
        return None
    print(f"Reading {len(file_list)} file(s) for group {group_name}...")
    df = pl.read_parquet(file_list)
    print(f"{group_name} loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
    return df

# === Read dataframes ===
sales_item_df = read_parquet_group(sales_item_files, "SALES ITEM")
sales_purchase_df = read_parquet_group(sales_purchase_files, "SALES PURCHASE")
sales_user_df = read_parquet_group(sales_user_files, "SALES USER")

# === Print dataset shapes ===
print("\nDataset summary:")
if sales_item_df is not None:
    print(f"sales_item_df: {sales_item_df.shape}")
if sales_purchase_df is not None:
    print(f"sales_purchase_df: {sales_purchase_df.shape}")
if sales_user_df is not None:
    print(f"sales_user_df: {sales_user_df.shape}")

# === Columns to drop ===
cols_to_drop = list(set([
    # SALES ITEM
    "is_deleted", "last_sync_date", "sync_error_message", "image_url",
    "description_new", "weight", "sync_status_id", "category_l1_id",
    "category_l2_id", "category_l3_id", "category_id", "manufacturer",
    # SALES PURCHASE
    "is_deleted", "event_type",
    # SALES USER
    "is_deleted", "sync_status_id", "last_sync_date", "sync_error_message"
]))

# === Drop columns directly ===
if sales_item_df is not None:
    existing_cols = [c for c in cols_to_drop if c in sales_item_df.columns]
    if existing_cols:
        sales_item_df = sales_item_df.drop(existing_cols)
    print(f"sales_item_df new shape: {sales_item_df.shape}")

if sales_purchase_df is not None:
    existing_cols = [c for c in cols_to_drop if c in sales_purchase_df.columns]
    if existing_cols:
        sales_purchase_df = sales_purchase_df.drop(existing_cols)
    print(f"sales_purchase_df new shape: {sales_purchase_df.shape}")

if sales_user_df is not None:
    existing_cols = [c for c in cols_to_drop if c in sales_user_df.columns]
    if existing_cols:
        sales_user_df = sales_user_df.drop(existing_cols)
    print(f"sales_user_df new shape: {sales_user_df.shape}")

# === Numeric columns for outlier capping ===
target_columns = {
    "SALES ITEM": ['price', 'creation_timestamp', 'gp'],
    "SALES PURCHASE": ['price'],
    "SALES USER": ['timestamp', 'install_date']
}

# === Function to cap outliers using IQR ===
def cap_outliers(df, cols):
    df_result = df.clone()
    for col in cols:
        if col not in df.columns:
            continue
        try:
            _ = df[col].to_numpy().astype(float)
        except:
            continue
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_iqr, upper_iqr = Q1 - 1.5*IQR, Q3 + 1.5*IQR
        df_result = df_result.with_columns(
            pl.when(pl.col(col) < lower_iqr).then(lower_iqr)
            .when(pl.col(col) > upper_iqr).then(upper_iqr)
            .otherwise(pl.col(col)).alias(col)
        )
    return df_result

# === Apply outlier capping ===
if sales_item_df is not None:
    sales_item_df = cap_outliers(sales_item_df, target_columns["SALES ITEM"])
if sales_purchase_df is not None:
    sales_purchase_df = cap_outliers(sales_purchase_df, target_columns["SALES PURCHASE"])
if sales_user_df is not None:
    sales_user_df = cap_outliers(sales_user_df, target_columns["SALES USER"])

print("\nOutlier capping complete for all datasets.")

# === Remove highly correlated columns ===
max_sample = 100_000

def remove_high_corr(df, threshold=0.85):
    num_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype in pl.NUMERIC_DTYPES]
    if len(num_cols) < 2:
        return df
    df_sample = df.select(num_cols)
    if df_sample.height > max_sample:
        df_sample = df_sample.sample(n=max_sample, seed=42)
    corr_matrix = df_sample.to_pandas().corr().abs()
    mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    high_corr_pairs = (
        corr_matrix.where(mask)
        .stack()
        .reset_index()
        .rename(columns={0: "correlation", "level_0": "feature_1", "level_1": "feature_2"})
        .query("correlation > @threshold")
    )
    to_drop = set(high_corr_pairs["feature_2"].tolist())
    if to_drop:
        df = df.drop(list(to_drop))
    return df

if sales_item_df is not None:
    sales_item_df = remove_high_corr(sales_item_df)

if sales_purchase_df is not None:
    sales_purchase_df = remove_high_corr(sales_purchase_df)

if sales_user_df is not None:
    sales_user_df = remove_high_corr(sales_user_df)

print("\nHigh correlation columns removed for all datasets.")


Found 83 parquet files in the folder.

Summary:
  sales_pers.item: 1 files
  sales_pers.purchase: 72 files
  sales_pers.user: 10 files

Reading 1 file(s) for group SALES ITEM...
SALES ITEM loaded: 27,332 rows × 34 columns
Reading 72 file(s) for group SALES PURCHASE...
SALES PURCHASE loaded: 35,729,825 rows × 16 columns
Reading 10 file(s) for group SALES USER...
SALES USER loaded: 4,573,964 rows × 18 columns

Dataset summary:
sales_item_df: (27332, 34)
sales_purchase_df: (35729825, 16)
sales_user_df: (4573964, 18)
sales_item_df new shape: (27332, 22)
sales_purchase_df new shape: (35729825, 14)
sales_user_df new shape: (4573964, 14)

Outlier capping complete for all datasets.

High correlation columns removed for all datasets.


In [3]:
#Cho em Hoe cute

print(sales_item_df.columns)
print(sales_purchase_df.columns)
print(sales_user_df.columns)

['p_id', 'item_id', 'price', 'category_l1', 'category_l2', 'category_l3', 'category', 'description', 'brand', 'created_date', 'updated_date', 'gender_target', 'age_group', 'item_type', 'gp', 'color', 'size', 'origin', 'volume', 'material', 'sale_status']
['timestamp', 'user_id', 'item_id', 'event_value', 'price', 'customer_id', 'created_date', 'updated_date', 'channel', 'payment', 'location', 'discount']
['customer_id', 'gender', 'location', 'province', 'membership', 'created_date', 'updated_date', 'region', 'location_name', 'install_app', 'install_date', 'district', 'user_id']
